In [1]:
print("hello world")

hello world


In [2]:
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 40.5 MB/s eta 0:00:00
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName("twitter_transformation")\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/01/04 19:16:41 WARN Utils: Your hostname, PcArthur resolves to a loopback address: 127.0.1.1; using 172.23.223.44 instead (on interface eth0)
25/01/04 19:16:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/04 19:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df = spark.read.json('../../datalake/twitter_datascience')

In [9]:
df.show()

+--------------------+--------------------+------------+
|                data|            includes|extract_data|
+--------------------+--------------------+------------+
|[{82, 62, 2024-12...|{[{2024-12-29T18:...|  2024-12-29|
|[{27, 41, 2024-12...|{[{2024-12-30T16:...|  2024-12-31|
|[{16, 91, 2024-12...|{[{2024-12-30T08:...|  2024-12-30|
+--------------------+--------------------+------------+

